In [11]:
import pandas as pd

# load data
data = pd.read_csv("./fnb_datav2.csv")

data = data.drop(columns=["item_descrip"])
data.head()

,idcol,interaction,int_date,item,page,tod,item_type,segment,beh_segment,active_ind
0,755,DISPLAY,17JAN2023,NONE,Screen1,Afternoon,ALL,segment3,B01,Semi Active
1,4521,DISPLAY,27FEB2023,NONE,Screen1,Afternoon,ALL,segment1,B07,Semi Active
2,4521,DISPLAY,18FEB2023,NONE,Screen1,Afternoon,ALL,segment1,B07,Semi Active
3,4521,DISPLAY,30JAN2023,NONE,Screen1,Morning,ALL,segment1,B07,Semi Active
4,4521,CLICK,05FEB2023,IBAB,Screen1,Afternoon,INSURE,segment1,B07,Semi Active


## Creating a interaction matgrix

In [60]:
# Pivot the DataFrame to create a user-item interaction matrix
# For simplicity, we'll use 'interaction' as the value to fill, you can change this as needed
# interaction_matrix = data.pivot_table(index='idcol', columns='item', values='interaction', aggfunc='count', fill_value=0)
# interaction_matrix

item,CABC,CACU,CAFB,CAFI,CAFM,CAFS,CAFU,CALI,CANL,CAPO,...,MMMC,MMSM,NACS,NAFW,NASD,NATR,NONE,SEVP,WHCR,XCFL
idcol,,,,,,,,,,,,,,,,,,,,,
755,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
6145,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,0,0,0
7125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25685663618,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,1
25693117918,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
25705258018,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


# EDA

In [17]:
data.nunique()

idcol          84375
interaction        3
int_date          88
item             104
page               2
tod                4
item_type          7
segment            4
beh_segment       50
active_ind         3
dtype: int64

# Trying Algos

In [89]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

# train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
interaction_matrix = data.pivot_table(index='idcol', columns='item', values='interaction', aggfunc='count', fill_value=0).T
print(interaction_matrix.shape)
# train_data = train_data.T
# test_data = test_data.T

matrix = csr_matrix(interaction_matrix)

knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(matrix)


(104, 84375)


NearestNeighbors(algorithm='brute', metric='cosine')

In [86]:
# print(test_data.shape)
# print(train_data.shape)
# test_data.head()

(10, 40000)
(360000, 10)


,23218,20731,39555,147506,314215,190913,296715,141482,49119,208005,...,218248,136839,145641,127025,237517,247748,192307,33042,135096,134650
idcol,9746386,8714418,16588662,71423806,366592771,102693842,312721492,67130979,20874426,125696269,...,139362138,64178344,70210615,58782653,173717536,193271861,104476434,13887348,63458744,63284129
interaction,CHECKOUT,CHECKOUT,DISPLAY,CLICK,DISPLAY,CLICK,CHECKOUT,CLICK,CLICK,DISPLAY,...,DISPLAY,CLICK,DISPLAY,CLICK,CHECKOUT,CLICK,CHECKOUT,CLICK,DISPLAY,DISPLAY
int_date,13FEB2023,10JAN2023,23FEB2023,05FEB2023,27MAR2023,24JAN2023,30MAR2023,28FEB2023,05MAR2023,24FEB2023,...,09JAN2023,11MAR2023,26MAR2023,23FEB2023,18MAR2023,26JAN2023,27JAN2023,31JAN2023,11FEB2023,05JAN2023
item,IBAB,FIWL,NONE,CBEL,NONE,EBQF,CCAI,CASD,EVGW,NONE,...,NONE,CTLN,NONE,CTLN,NAFW,FIHC,NATR,SEVP,NONE,NONE
page,Screen1,Screen1,Screen1,Screen1,Screen1,Screen1,Screen1,Screen2,Screen1,Screen1,...,Screen1,Screen2,Screen1,Screen2,Screen1,Screen1,Screen2,Screen1,Screen1,Screen1


In [95]:
distances, indices = knn.kneighbors(interaction_matrix.iloc[3,:].values.reshape(1, -1), n_neighbors = 5)
print(distances)
print(indices)


[[7.77156117e-16 9.01445266e-01 9.10965272e-01 9.11532129e-01
  9.15376451e-01]]
[[ 3 28 24 11 10]]


In [96]:
for i in range(len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(interaction_matrix.index[3]))
        
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, interaction_matrix.index[indices.flatten()[i]], distances.flatten()[i]))
        # break

Recommendations for CAFI:

1: CCLI, with distance of 0.9014452662221031:
2: CCAI, with distance of 0.9109652715872409:
3: CARF, with distance of 0.9115321291138396:
4: CARE, with distance of 0.9153764510715952:


In [63]:
print(len(distances.flatten()))
# print(test_data.index[indices.flatten()[0]])
print(indices.flatten()[0])

print(test_data.shape)

5
63
(104, 8438)
